# Decarbonization Projcet


# Model Implementation

#### Preparations

Import packages

In [1]:
import gurobipy as GRB
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

Load parameters


In [19]:
parameters = pd.read_excel("data.xlsx")
parameters

,source,number,capacity_plant_kw,fixed_cost_power_dollar_kW,fixed_cost_plant_dollar,operating_cost(dollar_kWh),revenues(dollor_kWh),co2(pounds_kWh),capacity_national_twh,capacity_national_tw,generation_national_twh,capacity_%,generation_%
0,coal,8,860112.500000,750,6.450844e+08,0.030,0.32,2.26,60.276684,0.006881,13.8012,0.079,0.062
1,natural gas,48,244968.750000,600,1.469813e+08,0.070,0.32,0.97,103.004460,0.011758,22.7052,0.135,0.102
2,CCGT,23,655143.478261,900,5.896291e+08,0.070,0.32,0.77,131.998308,0.015068,51.6432,0.173,0.232
3,nuclear,4,827450.000000,3100,2.565095e+09,0.035,0.32,0.00,28.993848,0.003310,25.5990,0.038,0.115
4,hydro,739,37715.832206,3100,1.169191e+08,0.010,0.32,0.00,244.158720,0.027872,56.9856,0.320,0.256
5,wind,256,58180.078125,3100,1.803582e+08,0.010,0.32,0.00,130.472316,0.014894,37.8420,0.171,0.170
6,solar,141,45094.326241,4500,2.029245e+08,0.010,0.32,0.00,55.698708,0.006358,7.7910,0.073,0.035


In [17]:
parameters[["number"]]

,number
0,8
1,48
2,23
3,4
4,739
5,256
6,141


In [7]:
# product_material = np.genfromtxt("F:\Learning\DABP\Assignment 1\Pb1_requirements.csv", dtype=float, delimiter=',', encoding='utf-8-sig')
# print("Shape: of product material", product_material.shape)
# resource = np.genfromtxt("F:\Learning\DABP\Assignment 1\Pb1_availability.csv", dtype=float, delimiter=',', encoding='utf-8-sig')
# print("Shape: of resource", resource.shape)
# demand = np.genfromtxt("F:\Learning\DABP\Assignment 1\Pb1_demand.csv", dtype=float, delimiter=',', encoding='utf-8-sig')
# print("Shape: of demand", demand.shape)
# cost = np.genfromtxt("F:\Learning\DABP\Assignment 1\Pb1_holdingcost.csv", dtype=float, delimiter=',', encoding='utf-8-sig')
# print("Shape: of cost", cost.shape)
# unit_profit = np.genfromtxt("F:\Learning\DABP\Assignment 1\Pb1_unitprofit.csv", dtype=float, delimiter=',', encoding='utf-8-sig')
# print("Shape: of profit", unit_profit.shape)
unit_emission = np.arange(7)
print("Shape of unit emission:", unit_emission.shape)
unit_cost = np.arange(7)
fixed_cost = np.arange(7)
yearly_demand = np.arange(10)
unit_price = 10
capacity = np.array(parameters[["capacity"]])
num_plant_start = np.array(parameters[["number"]]) # number of plants at year 0
budget_start = 9080 # million of euros

Shape of unit emission: (7,)


Set up index sets

In [8]:
years = range(yearly_demand.shape[0])  # 10
sources = range(unit_emission.shape[0])  # 7

#### Set up model

In [9]:
m = GRB.Model()

Decision variables

In [11]:
dv_num_plant = m.addVars(sources, years, vtype=GRB.INTEGER)  # 7*10
dv_add_plant = m.addVars(sources, years, vtype=GRB.INTEGER)  # 7*10
dv_minus_plant = m.addVars(sources, years, vtype=GRB.INTEGER)  # 7*10
dv_yearly_budget = m.addVars(years) # 10

Objective function

In [ ]:
# Primary Objective: carbon at year 10, set negative in accordance with the MAXIMIZE model sense
m.setObjectiveN(-sum(unit_emission[j] * capacity[j] * dv_num_plant[j] for j in sources), index=0, priority=10)
# Objective 2: total profit = revenue - fixed cost - operating cost
sum_revenue = sum(capacity[j]*dv_num_plant[j, i]*unit_price for j in sources for i in years)
sum_fixed_cost = sum(fixed_cost[j]*dv_add_plant[j, i] for j in sources for i in years)
sum_operating_cost = sum(unit_cost[j]*dv_num_plant[j, i]*capacity[j] for j in sources for i in years)
m.setObjectiveN(sum_revenue + sum_fixed_cost + sum_operating_cost, index=1, priority=5)
m.modelSense = GRB.MAXIMIZE

Constraints

In [ ]:
# Definition of S = s0 + sum(x)
for i in years:
    for j in sources:
        m.addConstr(dv_num_plant = num_plant_start[j] + sum(dv_add_plant[j, k] + dv_minus_plant[j, k] for k in range(0, i)), name="Definition of state for source {} year {}".format(j, i))

# Budget Definition
for i in years:
    if i == 0:
        m.addConstr(dv_yearly_budget <= budget_start)
    else:
        profit_prev_year = sum(capacity[j]*dv_num_plant[j, i-1]*unit_price for j in sources)
        m.addConstr(dv_yearly_budget <= profit_prev_year)

# Budget
for i in years:
    # fixed + operating cost
    yearly_cost = sum(fixed_cost[j]*dv_add_plant[j, i] + unit_cost[j]*dv_num_plant[j, i]*capacity[j] for j in sources)
    m.addVar(yearly_cost <= dv_yearly_budget[i])

# Demand
for i in years:
    m.addConstr(sum(capacity[j] * dv_num_plant[j, i] for j in sources) >= yearly_demand[i])

# Can't demolish clean energy


#### Solve the model

In [ ]:
# Solve
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 26520 rows, 15600 columns and 61260 nonzeros
Model fingerprint: 0x57c59e8f
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [7e-02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 7e+04]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 21002 rows and 410 columns
Presolve time: 0.07s
Presolved: 5518 rows, 15190 columns, 40132 nonzeros

Ordering time: 0.13s

Barrier statistics:
 AA' NZ     : 2.700e+04
 Factor NZ  : 2.543e+05 (roughly 10 MB of memory)
 Factor Ops : 3.066e+07 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.36958113e+08  4.13350654e+08  7.48e+04 4.32e+01  2.35e+05     0s
   1   8.73254128e+07 

It takes 0.67 seconds to solve the model.

In [ ]:
# Print optimal cost
print("Optimal net profit is ", m.objVal)


Optimal net profit is  35936031.39872105
      Week  Product     Produce        Sell  Inventory
0        1        1  286.700000  286.700000        0.0
1        1        2    0.000000    0.000000        0.0
2        1        3    0.000000    0.000000        0.0
3        1        4    0.000000    0.000000        0.0
4        1        5   38.963021   38.963021        0.0
...    ...      ...         ...         ...        ...
5195    52       96    0.000000    0.000000        0.0
5196    52       97    0.000000    0.000000        0.0
5197    52       98   54.837246   54.837246        0.0
5198    52       99    0.000000    0.000000        0.0
5199    52      100  337.830000  337.830000        0.0

[5200 rows x 5 columns]
